## PartA
In the 1st part, we are going to choose one sentence from each of the following texts and reconstruct them in a way that make sense.

In the 2nd part, we are going to choose 3 Paraphrasers from https://huggingface.co/ to paraphrase the text.

Then we are going to save the new text into .txt files and a benchmark so we can compare them later



Text1 =
Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in
our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the
professor, to show me, this, a couple of days ago. I am very appreciated the full support of the
professor, for our Springer proceedings publication

Text2 = 
During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?
Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance
and efforts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before
he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future
targets

From text1 we chose the sentence: Thank your message to show our words to the doctor, as his next contract checking, to all of us

From text2 we chose the sentence: Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation

To make sure the sentences we reconstruct are grammatically correct we are going to create a simple grammar in which we are going to parse the reconstructed sentences to at least determine whether or not they are grammatically correct.

In [77]:
import nltk
# Load the sentences from the files
with open("paraphrased_text/sentence1.txt", "r") as f1, open("paraphrased_text/sentence2.txt", "r") as f2:
    sentence1 = f1.read().strip().split('\n')
    sentence2 = f2.read().strip().split('\n')
# Simple context free grammar
grammar = nltk.CFG.fromstring("""
    S -> NP VP | VP | Adv S | S Conj S 
    NP -> Det N | Det N PP | Det Adj N | Det Adj N PP | NP PP | Pronoun | NP Conj NP | Adj NP | N | Det PP | N NP | Det Adj | N Adv
    PP -> P NP
    VP -> V NP | VP PP | V | Adv VP | VP Adv | V Pronoun | V VP | V S
    Det -> 'the' | 'a' | 'your' | 'our' | 'his' | 'all' | 'which' | 'some' | 'their'
    Adj -> 'next' | 'recent' | 'bit' | 'less' 
    Adv -> 'anyway' | 'really'  | 'even' | 'lately'  
    Pronoun -> 'us' | 'they' | 'i' | 'you' | 'he' | 'she' | 'who' | 'it' | 'them' | 'there'
    N -> 'doctor' | 'team' | 'message' | 'words' | 'contract' | 'days' | 'paper' | 'cooperation' | 'delay' | 'communication' | 'thoughts' | 'checking' | 'best' | 'delays'
    P -> 'to' | 'as' | 'for' | 'of' | 'at' | 'with' | 'in' | 'on' | 'by'
    V -> 'show' | 'believe' | 'tried' | 'thank' | 'helped' | 'assist' | 'convey' | 'will' | 'were' | 'did'
    Conj -> 'and' | 'though' | 'although'
""")
stop_words = [ '.', ',']
# Create a parser
parser = nltk.ChartParser(grammar)
# Parse the sentences.
# Fix the sentences
# to make them more grammatically correct
# and to match the grammar
def sentence_fixer(sentence):
    replace = {
        "your message": "you for your message,",
        "to show our words": "it helped convey our thoughts",
        "as his": "who will assist all of us with checking the",
        "checking, to all of us": ".",
        "anyway, ": "",
        ", although": " really did their best with the paper,",
        "bit delay": "though there were some delays",
        "at recent days,": "lately.",
        "they really tried best for paper and cooperation": ""
    }
    for key, value in replace.items():
        sentence = sentence.lower().replace(key, value)
    return sentence
# Save the fixed sentences to files
sentences_fixed1 = [sentence_fixer(sentence) for sentence in sentence1]
sentences_fixed2 = [sentence_fixer(sentence) for sentence in sentence2]
with open("paraphrased_text/fixed_sentence1.txt", "w") as f1, open("paraphrased_text/fixed_sentence2.txt", "w") as f2:
    f1.write(sentences_fixed1 if isinstance(sentences_fixed1, str) else "\n".join(sentences_fixed1))
    f2.write(sentences_fixed2 if isinstance(sentences_fixed2, str) else "\n".join(sentences_fixed2))

sentences_fixed = sentences_fixed1 + sentences_fixed2
# Parse the sentences
for sentence in sentences_fixed:
    print(f"Parsing: {sentence}")
    tokens = nltk.word_tokenize(sentence)
    tokens = [token.lower() for token in tokens if token not in stop_words]
    print(f"Tokens: {tokens}")
    
    try:
        tree = next(parser.parse(tokens))
        print(tree)
        tree.pretty_print()
    except StopIteration:
        print("No parse tree found.")
    print("\n")


Parsing: thank you for your message,
Tokens: ['thank', 'you', 'for', 'your', 'message']
(S
  (VP
    (V thank)
    (NP (NP (Pronoun you)) (PP (P for) (NP (Det your) (N message))))))
               S                      
               |                       
               VP                     
   ____________|___                    
  |                NP                 
  |       _________|___                
  |      |             PP             
  |      |      _______|____           
  |      NP    |            NP        
  |      |     |        ____|_____     
  V   Pronoun  P      Det         N   
  |      |     |       |          |    
thank   you   for     your     message



Parsing: it helped convey our thoughts to the doctor,
Tokens: ['it', 'helped', 'convey', 'our', 'thoughts', 'to', 'the', 'doctor']
(S
  (NP (Pronoun it))
  (VP
    (VP (V helped) (VP (V convey) (NP (Det our) (N thoughts))))
    (PP (P to) (NP (Det the) (N doctor)))))
                           S      

In [78]:
# ChatGPT Paraphraser
device = "cpu"
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)
def paraphrase(
    question,
    ParaphraseKeyword = False,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temp=0.9,
    max_length=128
):
    if ParaphraseKeyword:
        input_text = f'Paraphrase: {question}'
    else:
        input_text = f'{question}'
    input_ids = tokenizer(
        input_text,
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temp, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Load the text from the files
with open("paraphrased_text/text1.txt", "r") as f1, open("paraphrased_text/text2.txt", "r") as f2:
    text1 = f1.read().strip().split('\n')
    text2 = f2.read().strip().split('\n')

In [79]:
#ChatGPT (T5) Paraphraser
paraphrased_text1 = ""
for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text1 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text1:")
print(paraphrased_text1)



Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Our Chinese culture features a dragon boat festival today, designed to celebrate with all that is good and safe in our lives.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: May it be as my heartfelt thanks for the opportunity to have a wonderful time with you, too.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: Thank you for your message to show us what we said to the doctor as his next contract checking.


Original: I got this message to see the approved message.
Paraphrased: I received this message to view the authorized message.


Original: In fact, I have received the message from the professor, to show me, this, a couple of days ago.
Paraphrased: I received the message from the professor a few days ago, to show me this.


Original: I am very apprec

In [80]:
# ChatGPT (T5) Paraphraser
paraphrased_text2 = ""
for text in text2:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text2 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text2:")
print(paraphrased_text2)

# Save the paraphrased texts to files
with open("paraphrased_text/chatgpt_t5_paraphrased_text1.txt", "w") as f1, open("paraphrased_text/chatgpt_t5_paraphrased_text2.txt", "w") as f2:
    f1.write(paraphrased_text1)
    f2.write(paraphrased_text2)

Original: “During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?
Paraphrased: During our last discussion, I shared with him the news of the new submission we had been waiting for last autumn, but the changes were not complete as they did not include full feedback from the reviewer or editor.


Original: Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation.
Paraphrased: Despite experiencing some delays and less communication than in recent days, the team did well on paper and cooperated effectively.


Original: We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.
Paraphrased: The collective agreement and efforts made until the Springer link was finally announced last week are greatly

chatgpt_paraphraser:
  author={Vladimir Vorobev, Maxim Kuznetsov},
  title={A paraphrasing model based on ChatGPT paraphrases},
  year={2023}


In [ ]:
# Pegasus Paraphraser

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
tokenizer = PegasusTokenizer.from_pretrained("tuner007/pegasus_paraphrase")
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase").to(device)
paraphrased_text1 = ""
for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text1 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text1:")
print(paraphrased_text1)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Our Chinese culture has a dragon boat festival that is celebrated today.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: Hope you enjoy it as I wish.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: Thank you for your message, which will be shown to the doctor.


Original: I got this message to see the approved message.
Paraphrased: I received this message to see the approved one.


Original: In fact, I have received the message from the professor, to show me, this, a couple of days ago.
Paraphrased: I received a message from the professor, to show me what he was talking about.


Original: I am very appreciated the full support of the professor, for our Springer proceedings publication
Paraphrased: The professor supported the Springer proceedings publicati

In [82]:
# Pegasus Paraphraser
paraphrased_text2 = ""
for text in text2:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text2 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text2:")
print(paraphrased_text2)
# Save the paraphrased texts to files
with open("paraphrased_text/pegasus_paraphrased_text1.txt", "w") as f1, open("paraphrased_text/pegasus_paraphrased_text2.txt", "w") as f2:
    f1.write(paraphrased_text1)
    f2.write(paraphrased_text2)

Original: “During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?
Paraphrased: During our final discussion, I told him about the new submission that we were waiting for but it wasn't included with the other updates.


Original: Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation.
Paraphrased: The team tried their best for paper and cooperation despite the recent delay.


Original: We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.
Paraphrased: I think we should be grateful for the acceptance and efforts until last week, when the Springer link came.


Original: Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again.
Para

@misc{zhang2019pegasus,
    title={PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization},
    author={Jingqing Zhang and Yao Zhao and Mohammad Saleh and Peter J. Liu},
    year={2019},
    eprint={1912.08777},
    archivePrefix={arXiv},
    primaryClass={cs.CL}
}

Created by Arpit Rajauria https://x.com/arpit_rajauria

In [83]:
# Bart Paraphraser
from transformers import BartForConditionalGeneration, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('stanford-oval/paraphraser-bart-large')
model = BartForConditionalGeneration.from_pretrained('stanford-oval/paraphraser-bart-large').to(device)
paraphrased_text1 = ""
for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text1 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text1:")
print(paraphrased_text1)

Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased:  Today is the Dragon Boat Festival, and in Chinese culture we celebrate it with everything safe and beautiful in our lives.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased:  I hope you enjoy it as much as I do, my deepest wish.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased:  I thank you for your report showing our words to the doctor, as well as his next contractual check-up on all of us.


Original: I got this message to see the approved message.
Paraphrased:  I received this message so I could see the approved report.


Original: In fact, I have received the message from the professor, to show me, this, a couple of days ago.
Paraphrased:  Actually, I got a message from the professor to show me this some time ago.


Original: I am very appreciated the fu

In [84]:
# Bart Paraphraser
paraphrased_text2 = ""
for text in text2:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
        paraphrased_text2 += p + "\n"
        
    print("\n")
print("Final Paraphrased Text2:")
print(paraphrased_text2)

# Save the paraphrased texts to files
with open("paraphrased_text/bart_paraphrased_text1.txt", "w") as f1, open("paraphrased_text/bart_paraphrased_text2.txt", "w") as f2:
    f1.write(paraphrased_text1)
    f2.write(paraphrased_text2)

Original: “During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?
Paraphrased:  During our last discussion, I told him about the new submission we had been waiting for since last fall, but that update was confusing because it didn't include any full feedback from a reviewer or even an editor?


Original: Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation.
Paraphrased:  Anyway, I believe that the team, even if a little late and less communicative in recent days, has really tried their best to be paper-friendly and cooperative.


Original: We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.
Paraphrased:  I think we should all be grateful for his acceptance and effort until the Sprin

@inproceedings{xu-etal-2020-autoqa,
    title = "{A}uto{QA}: From Databases to {QA} Semantic Parsers with Only Synthetic Training Data",
    author = "Xu, Silei  and Semnani, Sina  and Campagna, Giovanni  and Lam, Monica",
    booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP)",
    month = nov,
    year = "2020",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/2020.emnlp-main.31",
    pages = "422--434",
}



## Using ChatGPT 
(https://chatgpt.com/share/68552e36-7b78-800f-8cb9-28b4dd15d294)

We paraphrased the 2 text and saved them as benchmark_chatgpt_text1.txt and benchmark_chatgpt_text2.txt

We will use them as a benchmark to compare the generated/reconstructed paraphrased texts

To evaluate the semantic consistency and transformation quality of the paraphrased texts, we used cosine similarity via TF-IDF vectorization. This method captures how closely the paraphrased versions align with the benchmark and original texts in terms of word usage and structure.

We compared each paraphrased text against the benchmark reconstruction.

The closer the cosine similarity is to 1.0 the better semantically aligned the texts are.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#Load original texts
with open("paraphrased_text/text1.txt", "r") as f1, open("paraphrased_text/text2.txt", "r") as f2:
    original_text1 = f1.read().strip()
    original_text2 = f2.read().strip()
# Load benchmark texts
with open("paraphrased_text/benchmark_chatgpt_text1.txt", "r") as f1, open("paraphrased_text/benchmark_chatgpt_text2.txt", "r") as f2:
    benchmark_text1 = f1.read().strip()
    benchmark_text2 = f2.read().strip()
# Load ChatGPT T5 paraphrased texts
with open("paraphrased_text/chatgpt_t5_paraphrased_text1.txt", "r") as f1, open("paraphrased_text/chatgpt_t5_paraphrased_text2.txt", "r") as f2:
    chatgpt_t5_text1 = f1.read().strip()
    chatgpt_t5_text2 = f2.read().strip()
# Load Pegasus paraphrased texts
with open("paraphrased_text/pegasus_paraphrased_text1.txt", "r") as f1, open("paraphrased_text/pegasus_paraphrased_text2.txt", "r") as f2:
    pegasus_text1 = f1.read().strip()
    pegasus_text2 = f2.read().strip()
# Load BART paraphrased texts
with open("paraphrased_text/bart_paraphrased_text1.txt", "r") as f1, open("paraphrased_text/bart_paraphrased_text2.txt", "r") as f2:
    bart_text1 = f1.read().strip()
    bart_text2 = f2.read().strip()
# Load fixed sentences
with open("paraphrased_text/fixed_sentence1.txt", "r") as f1, open("paraphrased_text/fixed_sentence2.txt", "r") as f2:
    fixed_sentence1 = f1.read().strip()
    fixed_sentence2 = f2.read().strip()

# Comput cosine similarity
def compute_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

for text1, text2, label in [
    (original_text1, benchmark_text1, "Benchmark and Original Text1"),
    (original_text2, benchmark_text2, "Benchmark and Original Text2"),
    (original_text1.splitlines()[2], benchmark_text1.splitlines()[2], "Benchmark Sentence1 and Original Sentence1"),
    (original_text2.splitlines()[1], benchmark_text2.splitlines()[1], "Benchmark Sentence2 and Original Sentence2"),
    (benchmark_text1, chatgpt_t5_text1, "ChatGPT T5 and Benchmark Text1"),
    (benchmark_text2, chatgpt_t5_text2, "ChatGPT T5 and Benchmark Text2"),
    (benchmark_text1, pegasus_text1, "Pegasus and Benchmark Text1"),
    (benchmark_text2, pegasus_text2, "Pegasus and Benchmark Text2"),
    (benchmark_text1, bart_text1, "BART and Benchmark Text1"),
    (benchmark_text2, bart_text2, "BART and Benchmark Text2"),
    (benchmark_text1.splitlines()[2], fixed_sentence1, "Fixed Sentence1 and Benchmark Sentence1"),
    (benchmark_text2.splitlines()[1], fixed_sentence2, "Fixed Sentence2 and Benchmark Sentence2"),
]:
    similarity = compute_cosine_similarity(text1, text2)
    print(f"Cosine Similarity between {label} : {similarity:.4f}")

Cosine Similarity between Benchmark and Original Text1 : 0.6790
Cosine Similarity between Benchmark and Original Text2 : 0.7174
Cosine Similarity between Benchmark Sentence1 and Original Sentence1 : 0.4202
Cosine Similarity between Benchmark Sentence2 and Original Sentence2 : 0.3301
Cosine Similarity between ChatGPT T5 and Benchmark Text1 : 0.6003
Cosine Similarity between ChatGPT T5 and Benchmark Text2 : 0.7301
Cosine Similarity between Pegasus and Benchmark Text1 : 0.5105
Cosine Similarity between Pegasus and Benchmark Text2 : 0.6726
Cosine Similarity between BART and Benchmark Text1 : 0.6018
Cosine Similarity between BART and Benchmark Text2 : 0.6827
Cosine Similarity between Fixed Sentence1 and Benchmark Sentence1 : 0.5103
Cosine Similarity between Fixed Sentence2 and Benchmark Sentence2 : 0.5920


We will be discussing the results further in the report, but as we can see the similarity between the original texts and the benchmarks is around 0.7 meaning that even though we reconstructed it to make it more understandable, it kept its general meaning.

We can also see that the reconstructed Text2 is way closer to Benchmark Text2, than the reconstructed Text1 to Benchmark Text1.
Which makes sense, since the original Text1 didn't make as much sense as the original Text2 making the reconstruction even harder.

Another thing we can see is that the reconstructed sentences we made, compared with the benchmark sentences, aren't that similar (only around 0.5-0.59 max). This is quite low which again indicated that the original sentences were grammaticaly/semantically ambiguous

## First Conclusion

Overall, we can see that all three models tried in a way to paraphrase as best as they could, although the meaning is still not 100% clear, it is way better than the original one.